# SparkContext - number of workers and lazy evaluation

## Checking the impact of number of workers
While initializing the `SparkContext`, we can specify number of worker nodes. Generally, it is recommended to have one worker per core of the machine. But it can be smaller or larger. In the following code, we will examine the impact of number of worker cores on some parallelized operation.

In [12]:
from time import time
from pyspark import SparkContext

In [13]:
for j in range(1,5):
    sc= SparkContext(master = "local[%d]"%(j))
    t0=time()
    for i in range(10):
        sc.parallelize([1,2]*10000).reduce(lambda x,y:x+y)
    print(f"{j} executors, time = {time()-t0}")
    sc.stop()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[2]) created by __init__ at <ipython-input-6-39fd847ec46a>:1 

#### We observe that it takes almost double time for 1 worker, and after that time reduces to a flat level for 2,3,4 workers etc. This is because this code run on a Linux virtual box using only 2 cores from the host machine. If you run this code on a machine with 4 cores, you will see benefit upto 4 cores and then the flattening out of the time taken. It also become clear that using more than one worker per core is not beneficial as it just does context-switching in that case and does not speed up the parallel computation.

## Showing the essence of _lazy_ evaluation
![](https://qph.fs.quoracdn.net/main-qimg-d49dcf35ecb7eecfc6e5b39493a0e086-c)

In [ ]:
sc = SparkContext(master="local[2]")

### Make a RDD with 1 million elements

In [ ]:
%%time
rdd1 = sc.parallelize(range(1000000))

### Some computing function - `taketime`

In [ ]:
from math import cos
def taketime(x):
    [cos(j) for j in range(100)]
    return cos(x)

### Check how much time is taken by `taketime` function

In [ ]:
%%time
taketime(2)

### Now do the `map` operation on the function

In [ ]:
%%time
interim = rdd1.map(lambda x: taketime(x))

#### How come each taketime function takes 45.8 us but the map operation with a 10000 element RDD also took similar time?<br><br>Because of _lazy_ evaluation i.e. nothing was computed in the previous step, just a plan of execution was made. The variable `interim` does not point to a data structure, instead it points to a plan of execution, expressed as a dependency graph. The dependency graph defines how RDDs are computed from each other.

### Let's see the "Dependency Graph" using `toDebugString` method

In [ ]:
print(interim.toDebugString().decode())

![](https://raw.githubusercontent.com/tirthajyoti/Spark-with-Python/master/Images/RDD_dependency_graph.PNG)

### The actual execution by `reduce` method

In [ ]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

In [ ]:
1000000*31e-6

#### It is less than what we would have expected considering 1 million operations with the `taketime` function. This is the result of parallel operation of 2 cores.

### Now, we have not saved (materialized) any intermediate results in `interim`, so another simple operation (e.g. counting elements > 0) will take almost same time

In [ ]:
%%time
print(interim.filter(lambda x:x>0).count())

## Caching to reduce computation time on similar operation (spending memory)

### Run the same computation as before with `cache` method to tell the dependency graph to plan for caching

In [ ]:
%%time
interim = rdd1.map(lambda x: taketime(x)).cache()

In [ ]:
print(interim.toDebugString().decode())

In [ ]:
%%time
print('output =',interim.reduce(lambda x,y:x+y))

### Now run the same `filter` method with the help of cached result

In [ ]:
%%time
print(interim.filter(lambda x:x>0).count())

#### This time it took much shorter time due to cached result, which it could use to compare to 0 and count easily.